In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1747521064076, experiment_id='2', last_update_time=1747521064076, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747521029154, experiment_id='1', last_update_time=1747521029154, lifecycle_stage='active', name='my-brand-new-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1747519090377, experiment_id='0', last_update_time=1747519090377, lifecycle_stage='active', name='Default', tags={}>]

In [4]:
client.create_experiment(name = "my-cool-experiment")

'3'

In [7]:
from mlflow.entities import ViewType

runs = client.search_runs(
experiment_ids = '2',
filter_string='',
max_results= 5,
order_by = ['metrics.rmse ASC']
)

In [ ]:
for run in runs:
    print(f'run_id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

'run_id: d3b1d08055064ce9b55944d9df946a14, 'rmse: 6.3056'
'run_id: ed343f43a43e4e769e25f8fbb25cf66b, 'rmse: 6.3095'
'run_id: 7f7d298a004e49c982477d847a0edc2f, 'rmse: 6.3107'
'run_id: 442a5c6da10a440ab5fab88fd654e6d3, 'rmse: 6.3137'
'run_id: c18de08d074a4381939764cdc5f54b71, 'rmse: 6.3190'


In [17]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_uri = 'c18de08d074a4381939764cdc5f54b71'
run_id = f'runs:/{run_uri}/model'

result = mlflow.register_model(run_id, 'nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


In [ ]:
client.search_registered_models()

[<RegisteredModel: aliases={'challenger': 2, 'champion': 1}, creation_timestamp=1748307595453, description='The NYC predictor for trip duration', last_updated_timestamp=1748309651378, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748309651378, current_stage='None', description=None, last_updated_timestamp=1748309651378, name='nyc-taxi-regressor', run_id='c18de08d074a4381939764cdc5f54b71', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2/c18de08d074a4381939764cdc5f54b71/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1748309625810, description=None, last_updated_timestamp=1748309625822, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748309625822, current_stage='None', description=None, last_updated_timestamp=1748309625822, name='nyc_taxi_regressor', run_id='c18de08d074a4381939764cdc5f54b71', 

In [25]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)


/tmp/ipykernel_53134/1035904062.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)
